In [1]:
import pandas as pd
from plotnine import *

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='svg'

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
sample=pd.read_csv('D:/Dataset/product-hunt-prouducts.csv')
sample['release_date']=pd.to_datetime(sample['release_date'])
sample['product_ranking']=sample['product_ranking'].map({1.0:'Rank #1',2.0:'Rank #2',3.0:'Rank #3',4.0:'Rank #4',5.0:'Rank #5'})
sample=sample[sample['product_ranking'].isnull()==False]

In [18]:
types=['PHOTOGRAPHY','PRODUCTIVITY','DESIGN TOOLS','WEB APP']

In [19]:
category=sample['category_tags']
category=category.apply(lambda x:eval(x))#.isnull()
category=category.apply(lambda x: 'Other' if x==list() else x[0])#.iloc[:6]#.isnull()
category=category.apply(lambda x: x if x in types else 'other')
sample['category']=category

In [20]:
Sample=sample[sample['category']!='other']

In [21]:
result=[]
groupby_rank=sample.groupby(sample['category'])
for name,group in groupby_rank:
    result.append(group.sort_values(by='upvotes',ascending=False)[:5])
Top=pd.concat(result)
SERIES=pd.Series(object)
SERIES=pd.Series('others',index=Sample.index)
for Index in Top.index:SERIES[Index]='Tops'
Sample['attribute']=SERIES
Top=Top[Top['category']!='other']

In [68]:
p=(
    ggplot(aes( x='release_date',y='upvotes'), Sample)#[:1000])
    
    #把数据中的点映射进去
    + geom_point( 
        aes(color='attribute',fill='attribute',alpha='attribute'),
        #aes(color='upvotes'),
        size = 2.5, 
        #alpha = 0.95,
        shape = 'o', stroke = .6,
        
    )
    
    
    + scale_color_manual(values = ('#3B293D', '#942534')) 
    + scale_fill_manual(values = ('#DE763B', '#F9F2D6'))
    
    + scale_alpha_manual(values=(1,0.9))

    #+ geom_smooth(span = .5, alpha = 0.75, color = "#900046",method='lm')#,method='loess')
    
    + geom_label(
        aes(x = 'release_date', y = 'upvotes', label = 'name'),
        data = Top, #设定你要画label的数据，index一定要按顺序，不然报错 #rank_one_sample[:10], #rank_one_sample.sort_values(by='upvotes',ascending=False)[:1],
        alpha=0.7,
        ha='left',
        size = 10, #整个矩形的大小
        label_size = 0.3, #矩形边框的大小
        nudge_x=100
    )
    
    + scale_x_datetime(expand = (0, 0),labels=[2014,'', 2016,'', 2018, '',2020,'']) 
    
    + facet_wrap('category', nrow = 1) #分成若干个子图
    
    + coord_cartesian(expand = True)#, clip = "off")
    
    + theme_minimal(base_family = "Goudy Old Style")
    
    + theme(
        figure_size=(16, 7),
        legend_position = "right",
        strip_text = element_text(face = "bold"),
        axis_text_x = element_text(size = 8),
        #axis_text_y = element_text(size = 8),
        axis_title_y=element_text(size=12,angle=0),
        plot_title = element_text(size = 14, face = "bold"),
        plot_background=element_rect(fill='#f2f2f2', alpha=.3),
        #element_line='dashed'
        #strip_margin_y=0.5,
        #strip_margin_x=0.1,
    )

    + labs(
        y = '''
        Number Of Upvotes                           
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        ''',
        x = '''
        Release Date (Year)
        ''',
        title = '''
        Product Hunt
        ''',
        caption="Source : components.one | Graphic : LZU DS Team17",
    )
)

In [69]:
p.save(filename='rebuild1.svg')